## Paralleling Recurrent Convolutional Neural Network

This is an implementation of the model presented in Feng et al. "Music genre classification with paralleling recurrent convolutional neural network."

In [0]:
import numpy as np

import keras
from keras.models import Model
from keras.layers import Input, Dense, Bidirectional, Dropout, Activation, GRU
from keras.layers import Conv2D, concatenate, MaxPooling2D, Flatten, Lambda
from keras import backend as K

from keras.optimizers import Adam, RMSprop

from sklearn.metrics import classification_report

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Retrieve all of the features and labels

In [0]:
npzfile = np.load('drive/My Drive/CSCI/CSCI4622/data/melspects_128.npz', allow_pickle=True)

In [0]:
X_train, y_train = npzfile['X_train'], npzfile['y_train']
X_test, y_test = npzfile['X_test'], npzfile['y_test']

In [0]:
X_train.shape

(900, 128, 640)

## Building the Neural Net

In [0]:
def PRCNN(X_shape, nb_classes):
    
    # Input
    inputs = Input(shape=X_shape)
    
    # CNN Block
    conv1 = Conv2D(filters=16, kernel_size=(1,3), strides=1, padding='valid', 
                   activation='relu')(inputs)
    
    pool11 = MaxPooling2D((2, 2), strides=(2,2))(conv1)
    
    conv2 = Conv2D(filters=32, kernel_size=(1,3), strides=1, padding='valid', 
                   activation='relu')(pool11)
    
    pool12 = MaxPooling2D((2, 2), strides=(2,2))(conv2)
    
    conv3 = Conv2D(filters=64, kernel_size=(1,3), strides=1, padding='valid', 
                   activation='relu')(pool12)
    
    pool13 = MaxPooling2D((2, 2), strides=(2,2))(conv3)
    
    conv4 = Conv2D(filters=128, kernel_size=(1,3), strides=1, padding='valid', 
                   activation='relu')(pool13)
    
    pool14 = MaxPooling2D((4, 4), strides=(4,4))(conv4)
    
    conv5 = Conv2D(filters=64, kernel_size=(1,3), strides=1, padding='valid', 
                   activation='relu')(pool14)
    
    pool15 = MaxPooling2D((4, 4), strides=(4,4))(conv5)

    flatten1 = Flatten()(pool15)
    
    # BGRU-RNN block
    pool21 = MaxPooling2D((1,2), strides=(1,2))(inputs)
    
    # Remove channel axis so we can pass into Bidirectional GRU layer
    squeezed = Lambda(lambda x: K.squeeze(x, axis=-1))(pool21)
    
    # Bidirectional GRU
    bigru_rnn = Bidirectional(GRU(128), merge_mode='concat')(squeezed)
    
    # Concat Output
    concat = concatenate([flatten1, bigru_rnn])
    
    # Softmax Output
    output = Dense(nb_classes, activation='softmax')(concat)
    
    model_output = output
    model = Model(inputs=inputs, outputs=[output])
    
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.0005), metrics=['accuracy'])
    
    print(model.summary())
    return model

In [0]:
nb_classes = len(np.unique(y_train))
nb_epochs = 25
batch_size = 4

# Convert to one hot encoding
y_train_one_hot = np.zeros((y_train.size, nb_classes))
y_train_one_hot[range(y_train.size),y_train] = 1

# Need to add a channel dim for the convolution layers
X_train_expanded = np.expand_dims(X_train, axis=-1)
X_test_expanded = np.expand_dims(X_test, axis=-1)

X_shape = X_train_expanded.shape[1:]

model = PRCNN(X_shape, nb_classes)

# Fit data to model
history = model.fit(X_train_expanded, y_train_one_hot, batch_size=batch_size, 
                    epochs=nb_epochs, validation_split=0.2)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 640, 1)  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 128, 638, 16) 64          input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_7 (MaxPooling2D)  (None, 64, 319, 16)  0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 64, 317, 32)  1568        max_pooling2d_7[0][0]            
____________________________________________________________________________________________

In [0]:
preds = model.predict(X_test_expanded)
y_pred = np.argmax(preds, axis=1)

In [0]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.08      0.14        12
           1       0.73      0.85      0.79        13
           2       0.55      0.60      0.57        10
           3       0.29      0.80      0.42         5
           4       0.67      0.33      0.44         6
           5       0.61      0.73      0.67        15
           6       0.60      0.90      0.72        10
           7       1.00      0.44      0.62         9
           8       0.58      0.50      0.54        14
           9       0.33      0.33      0.33         6

    accuracy                           0.57       100
   macro avg       0.59      0.56      0.52       100
weighted avg       0.61      0.57      0.55       100

